In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [ ]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
def get_coords(multiplication_factor,t_slab_x,t_slab_y,t_slab_z):
    x_size,y_size,z_size = t_slab_x/multiplication_factor,t_slab_y/multiplication_factor,t_slab_z
    coordinates_slabs=[]
    for i in range(multiplication_factor):
        for j in range(multiplication_factor):
            center_x = (i - ( multiplication_factor/ 2) + 0.5) * x_size
            center_y = (j - ( multiplication_factor/ 2) + 0.5) * y_size
            center_z = 0  # All cubes are centered on the z=0 plane
            coord_item = {
                    "X": (center_x - x_size/2, center_x + x_size/2),
                    "Y": (center_y - y_size/2, center_y + y_size/2),
                    "Z": (-z_size/2, z_size/2),
                    "center": (center_x, center_y, center_z)
                    }
            
            coordinates_slabs+=[coord_item]

    return coordinates_slabs

In [ ]:
# Function to create a solid cylinder with specified center coordinates
def create_cylinder_from_ends(top_center, bottom_center, radius):
    # Calculate height of the cylinder
    height = np.linalg.norm(np.array(top_center) - np.array(bottom_center))
    bottom_center=bottom_center

    # Calculate the vector direction of the cylinder
    axis_direction = np.array(top_center) - np.array(bottom_center)
    axis_direction /= np.linalg.norm(axis_direction) #Unitary vector to calculate rotation angle 

    # Calculate the rotation matrix to align cylinder with the given axis direction
    z_axis = np.array([0, 0, 1])
    rotation_axis = np.cross(z_axis, axis_direction)
    rotation_angle = np.arccos(np.dot(z_axis, axis_direction))
    rotation_matrix = mesh.Mesh.rotation_matrix(rotation_axis, rotation_angle)
    # Create a 4x4 identity matrix
    matrix_4x4 = np.eye(4)

    # Insert the 3x3 matrix into the top-left corner of the new matrix
    matrix_4x4[:3, :3] = np.copy(rotation_matrix)

    # Set the fourth column and fourth row for homogeneous transformation
    # The typical homogeneous transformation uses [0, 0, 0, 1] for the last row and column
    matrix_4x4[3, :3] = [0, 0, 0]  # Fourth row
    matrix_4x4[:3, 3] = [0, 0, 0]  # Fourth column
    matrix_4x4[3, 3] = 1           # Bottom-right corner remains 1

    
   
    cylinder_center = tuple((np.array(top_center) + np.array(bottom_center))/2)
    trans = td.Transformed.translation(cylinder_center[0],cylinder_center[1],cylinder_center[2])
    transformed = td.Transformed(geometry=td.Cylinder(center=(0,0,0), radius=radius, length=height),transform=trans@matrix_4x4)

    return transformed

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a = 5/3
run = True

In [ ]:
# lambdas = a/np.array([0.09,0.41]) 
# freqs_monitor = td.C_0/(a/np.array([0.09,0.12,0.15,0.20,0.21,0.23]))
lambdas = a/np.array([0.35,0.5]) 
folder_path = r"H:\phd stuff\tidy3d\structures\08_13_2024 Luis H5 18.01a\sample 2"
project_name = "09_03_2024 Freq Monitors Analysis Divergence"
postprocess_results = []
runtime_ps = 30e-12
min_steps_per_lambda = 15
for direction in ["z"]: 
    for filename in os.listdir(folder_path):
     for perm in [13]:
        print(filename)
        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=18*a,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda, permittivity=perm,
                                           scaling=1,shuoff_condtion=1e-15, verbose=True, 
                                           monitors=["flux"], freqs=25, 
                                           cut_condition=1.0, source="planewave", absorbers=120, use_permittivity=False,
                                           ref_only=True
                                           )

In [ ]:
file_path =  r"H:\phd stuff\tidy3d\structures\End2EndFiles\RCP 18.01 End Points\1_sample_L18_lines_cut.dat"
data = []
with open(file_path, 'r') as file:
    for line in file: 
        row = [float(value) for value in line.split()]
        data += [np.array(row)]

data = np.array(data)

tops_centered = (data[:,:3]-0.5)*structure_1.t_slab_z
bottoms_centered = (data[:,3:]-0.5)*structure_1.t_slab_z

radius = 0.25


cyl_group = []



coords = get_coords(structure_1.multiplication_factor,structure_1.t_slab_x,structure_1.t_slab_y,structure_1.t_slab_z)


for k,coord in enumerate(coords):
    tops = np.copy(tops_centered)
    bottoms = np.copy(bottoms_centered)
    tops[:,0]+=coord["center"][0]
    tops[:,1]+=coord["center"][1]
    tops[:,2]+=coord["center"][2]
    bottoms[:,0]+=coord["center"][0]
    bottoms[:,1]+=coord["center"][1]
    bottoms[:,2]+=coord["center"][2]
    for cil,item in enumerate(tops):
         cyl_group.append(create_cylinder_from_ends(tops[cil], bottoms[cil], radius))

    
   


In [ ]:
sim = structure_1.sim


In [ ]:
medium=td.Medium(permittivity=structure_1.permittivity_value)
eps_eff  = ((2+structure_1.permittivity_value )+ 2*0.3*(structure_1.permittivity_value-1))/((2+structure_1.permittivity_value)-0.3*(structure_1.permittivity_value-1))
medium_eff=td.Medium(permittivity=eps_eff)
structure = td.Structure(geometry=td.GeometryGroup(geometries=cyl_group),  medium=medium)
slices = np.array([18])*a
# slices = np.array([1.2,1.5,2,3,5,8,12,15,18])*a
# slices = np.array([0.1,0.3,0.5,0.9,1.0])*a

add_ref = False
for cut in slices: 
  box_in = td.Box(center=[0, 0,-(structure_1.Lz/2+structure_1.t_slab_z/2)/2], size=[td.inf, td.inf, structure_1.Lz / 2-structure_1.t_slab_z/2])
  box_out = td.Box(center=[0, 0,(structure_1.Lz/2+structure_1.t_slab_z/2)/2], size=[td.inf, td.inf, structure_1.Lz / 2-structure_1.t_slab_z/2 ])
  struct_in = td.Structure(geometry=box_in, medium=medium_eff)
  struct_out = td.Structure(geometry=box_out, medium=medium_eff)

  
  cube_eff = td.Structure(geometry= td.Box(
                        center=(0, 0 ,cut/2),
                        size=(
                                structure_1.t_slab_x if structure_1.direction == "x"  else td.inf, 
                                structure_1.t_slab_y if structure_1.direction == "y"  else td.inf, 
                                structure_1.t_slab_z-cut if structure_1.direction == "z"  else td.inf
                              ),
                    ),medium=medium_eff)
  

  ##########Monitors################
  freq_monitorField = td.FieldMonitor(
                center = (0,0,0),
                size = (
                    structure_1.t_slab_x,
                    structure_1.t_slab_y,
                    structure_1.t_slab_z,
                    ),
                    fields=["Ex","Ey","Ez"],
                    freqs =structure_1.monitor_freqs,
                    name="freq_monitorField",
                    
                )
  

  eps_monitorField = td.PermittivityMonitor(
                center = (0,0,0),
                size = (
                    structure_1.t_slab_x,
                    structure_1.t_slab_y,
                    structure_1.t_slab_z,
                    ),
                    freqs =structure_1.monitor_freqs,
                    name="eps_monitorField",
                    
                )
  ######################################







  if structure_1.t_slab_z-cut==0:
    sim = sim.copy(update={"structures":[structure,struct_in, struct_out],"monitors":list(structure_1.sim.monitors)+[freq_monitorField]})
  else:
    sim = sim.copy(update={"structures":[structure,struct_in, struct_out,cube_eff],"monitors":list(structure_1.sim.monitors)+[freq_monitorField]})
  sim_name = f"RCP perm {perm} runtime {runtime_ps}ps nu {a/lambdas[0]} - {a/lambdas[1]} Size - {cut/a}a"
  print(sim_name)
  sim.plot_3d()
  # fig, ax = plt.subplots(1, tight_layout=True, figsize=(16, 8))
  # freqs_plot = (structure_1.freq_range[0], structure_1.freq_range[1])
  # sim.plot_eps(x=0, freq=freqs_plot[0], ax=ax)
  # plt.show()
  # print(f"Time Steps = {sim.num_time_steps}\n"
  #       f"Grid Points = {sim.num_cells*1e-6} million\n")
  # id =web.upload(sim,task_name=sim_name, verbose=True)
  # web.estimate_cost(id)
  # web.delete(id)
  
  if run:
        id0 = ""
        if add_ref:
            print("running ref...")
            id0 = web.upload(sim.copy(update={"structures":[],"medium":medium_eff}), folder_name=project_name,task_name=sim_name+'_0', verbose=True)
            web.start(task_id = id0)
            web.monitor(task_id=id0,verbose=True)
            add_ref=False

        id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
        web.start(task_id = id)
        web.monitor(task_id = id,verbose=True)
    
        ids = (id0 if id0 else '') +'\n' + id
        incidence_folder = "z_incidence"
        file_path = rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}/{sim_name}.txt"
        # Check if the folder exists
        if not os.path.exists( rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}"):
            os.makedirs(rf"H:\phd stuff\tidy3d\data/{project_name}/{incidence_folder}")
            print(f"Folder '{project_name}/{incidence_folder}' created successfully.")

        # Open file in write mode
        with open(file_path, "w") as file:
            # Write the string to the file
            file.write(ids)

